#  Lexical Resource (LR) for Annotation

#### Imports

In [1]:
# interface
from tqdm import tqdm
from dsutils.de.files import dbg

## Files and filesystem
import os
import json
import csv
import glob

## Data management
import numpy as np
import pandas as pd

## DS tools
import re
from types import NoneType
from dsutils.de.files import describe_csv, get_csv_head, xls_to_csv, get_data_path, get_datafile_path
from dsutils.nlp.language import get_lang_code

[nltk_data] Error loading words: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


#### Paths

In [2]:
## input
db_paths = dict(
    eppo_com = get_datafile_path('2022-09-02_COMMONnames_EPPO_OQ.xlsx'),
    eppo_sci = get_datafile_path('2022-09-02_SCIENTIFICnames_EPPO_OQ.xlsx'),
    efsa = get_datafile_path('EFSA-keyword-match/FichierMotsClesMagaliLarenaudie.csv'),
    ncbi = get_datafile_path('taxa+id_full.txt'),
)

glossary_paths = dict(
    efsa_glossary = get_datafile_path('efsa_glossary.csv'),
    eppo_glossary = get_datafile_path('eppo_glossary.csv'),
)

## Output
data_path = get_data_path()
full_eppo_path = os.path.join(data_path, 'eppo_glossary.csv')
annotation_path = os.path.join(data_path, 'annotation_glossary')
anno_gloss_path = os.path.join(data_path, 'annotation_glossary.csv')
LR_filepath = 'LR.csv'

FileNotFoundError: [Errno 2] No such file or directory: '/home/elubrini/GitHub/bio-corpus-translation/data/2022-09-02_COMMONnames_EPPO_OQ.xlsx'

#### Definitions

Defining functions for preprocessing of original data

In [ ]:
def PHT_to_lower_taxon(PHT_code):
    taxon = re.sub(r'(?<!^)(?=[A-Z])', ' ', PHT_code[:-4]).lower()
    return taxon
# print('AcalymmaVittatum-PHT:',PHT_to_lower_taxon('AcalymmaVittatum-PHT')) # test
assert PHT_to_lower_taxon('AcalymmaVittatum-PHT') == 'acalymma vittatum'

def to_pattern(syn):
    r = syn.replace('+', ' ')
    r = r.replace('_', '\\w')
    if r.endswith('%'):
        r = r[:-1] + '\\w*'
    r = r.replace('% ', '\\w* ')
    r = r.replace('%', '\\S*\\s?')
    return r
assert to_pattern(r'chrysom_le%+ray_e+du+concombre') == 'chrysom\wle\w* ray\we du concombre'

#### Parameters

In [ ]:
ncbi_sep = '\t'

## name of the desired coluimns in the LR
anno_gloss_column_names = ['EPPO_CODE', 'EFSA_PHT', 'NCBI_TAXID', # 3 partial keys to relative dataset
                           'preferred_name', 'synonym', 're',# terms
                           'ds_language', 'match_language'] # languages

In [ ]:
## column in each of the original datasets corresponding to the output LR column 
cc = dict(
    eppo_com = ['CodeEOPP', None, None,
                'PreferredName', 'CommonName', None,
                ['Language', get_lang_code], None], # [obj (col name), fun] tuple when information is extracted via a function
    eppo_sci = ['CodeEOPP', None, None,
                'PreferredName', 'OtherScientificNames', None,
                None, None],
    efsa = [None, 'Category (pest name)', None,
                ['Category (pest name)', PHT_to_lower_taxon], None,  ['Keywords', to_pattern],
                'Unnamed: 3', None],
    ncbi = [None, None, 'ncbi:2',
            'Bacteria.1', 'Bacteria', None,
            None, None],
    )

#### Check Data

In [ ]:
## USE SMALLER VERSION OF NCBI
ncbi_small = pd.read_csv(db_paths['ncbi'], on_bad_lines='skip', sep='\t').head(100)
ncbi_small.head()
ncbi_small.to_csv(db_paths['ncbi'])

In [ ]:
for path in db_paths.values():
    display(describe_csv(path))

,CommonName,Language,CodeEOPP,PreferredName,AuthorityPreferredName
count,1276,1276,1324,1324,1142
unique,1235,38,239,239,169
top,citrus leprosis,English,HELIZE,Helicoverpa zea,(Boddie)
freq,4,477,30,30,30


,OtherScientificNames,Authority,CodeEOPP,PreferredName,AuthorityPreferredName
count,616,453,664,664,502
unique,611,324,239,239,169
top,Tomato leaf curl New Delhi begomovirus,Corbett,BEMITA,Bemisia tabaci,(Gennadius)
freq,2,6,16,16,16


,Category (pest name),Keywords,Unnamed: 2,Unnamed: 3
count,7558,7558,21,3
unique,1168,7393,5,3
top,Cronartium-PHT,margarodes,Au/Cabi,Fr
freq,85,3,16,1


,Unnamed: 0
count,100.000000
mean,49.500000
std,29.011492
min,0.000000
25%,24.750000
50%,49.500000
75%,74.250000
max,99.000000


In [ ]:
pd.read_csv(db_paths['ncbi'])


,Unnamed: 0,Bacteria,ncbi:2,Bacteria.1,/ncbi:1/ncbi:131567/ncbi:2,NP,superkingdom,Unnamed: 6,Unnamed: 7
0,0,bacteria,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NNS,superkingdom,NaN,NaN
1,1,eubacteria,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NN,superkingdom,NaN,NaN
2,2,Monera,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NP,superkingdom,NaN,NaN
3,3,Procaryotae,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NP,superkingdom,NaN,NaN
4,4,Prokaryotae,ncbi:2,Bacteria,/ncbi:1/ncbi:131567/ncbi:2,NP,superkingdom,NaN,NaN


## Create LR

### (1) Preprocess original datasets

Extract relevant information from each dataset and store it in table form (one `csv` per original dataset)

In [ ]:
## Display examples of input datasets
display(pd.read_csv(db_paths['eppo_sci']).describe())
pd.read_csv(db_paths['eppo_com']).describe()

,OtherScientificNames,Authority,CodeEOPP,PreferredName,AuthorityPreferredName
count,616,453,664,664,502
unique,611,324,239,239,169
top,Tomato leaf curl New Delhi begomovirus,Corbett,BEMITA,Bemisia tabaci,(Gennadius)
freq,2,6,16,16,16


,CommonName,Language,CodeEOPP,PreferredName,AuthorityPreferredName
count,1276,1276,1324,1324,1142
unique,1235,38,239,239,169
top,citrus leprosis,English,HELIZE,Helicoverpa zea,(Boddie)
freq,4,477,30,30,30


In [ ]:
db = dict()
for db_name, columns in cc.items():
    db[db_name] = dict()
    cc_zip = zip(columns, anno_gloss_column_names)
    for orig_col, targ_col in cc_zip:
        if isinstance(orig_col, str):
            def fun(x):
                return x
        elif isinstance(orig_col,NoneType):
            continue
        else:
            fun = orig_col[1]
            orig_col = orig_col[0]    
            
        # sep = tab (for ncbi)
        if 'ncbi' in db_name:
            ds_col = pd.read_csv(db_paths[db_name], on_bad_lines='skip', sep='\t')[orig_col]
        else:    
            ds_col = pd.read_csv(db_paths[db_name])[orig_col]
            
        db[db_name][targ_col] = list(map(fun, ds_col)) # new_db col = old_db col
    pd.DataFrame.from_dict(db[db_name]).to_csv(os.path.join(annotation_path, db_name+'_table.csv'), index=False)


KeyError: 'ncbi:2'

### (2) Join Datasets

#### Merge, Case Insensitive

In [ ]:
annotation_path = os.path.join(data_path, 'annotation_glossary')
all_files = glob.glob(os.path.join(annotation_path, "*.csv"))
print(all_files)
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
print(df.shape)
df.head()

['/home/elubrini/GitHub/bio-corpus-translation/data/annotation_glossary/re_LR.csv', '/home/elubrini/GitHub/bio-corpus-translation/data/annotation_glossary/eppo_com_table.csv', '/home/elubrini/GitHub/bio-corpus-translation/data/annotation_glossary/eppo_sci_table.csv', '/home/elubrini/GitHub/bio-corpus-translation/data/annotation_glossary/efsa_table.csv', '/home/elubrini/GitHub/bio-corpus-translation/data/annotation_glossary/annotation_LR.csv', '/home/elubrini/GitHub/bio-corpus-translation/data/annotation_glossary/ncbi_table.csv', '/home/elubrini/GitHub/bio-corpus-translation/data/annotation_glossary/LR.csv']


/tmp/ipykernel_27441/1111597208.py:4: DtypeWarning: Columns (1,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
/tmp/ipykernel_27441/1111597208.py:4: DtypeWarning: Columns (1,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
/tmp/ipykernel_27441/1111597208.py:4: DtypeWarning: Columns (1,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


(17419180, 7)


,preferred_name,EPPO_CODE,synonym,ds_language,EFSA_PHT,re,NCBI_TAXID
0,'Abelmoschus esculentus' bunchy top phytoplasma,NaN,'Abelmoschus esculentus' bunchy top phytoplasma,NaN,NaN,NaN,ncbi:926989
1,'Acacia arabica' phytoplasma,NaN,'Acacia arabica' phytoplasma,NaN,NaN,NaN,ncbi:2849181
2,'Acacia farnesiana' leaflet phytoplasma,NaN,'Acacia farnesiana' leaflet phytoplasma,NaN,NaN,NaN,ncbi:2654099
3,'Acacia mangium' little leaf and yellowing phy...,NaN,'Acacia mangium' little leaf and yellowing phy...,NaN,NaN,NaN,ncbi:2530013
4,'Acacia melanoxylon' phytoplasma,NaN,'Acacia melanoxylon' phytoplasma,NaN,NaN,NaN,ncbi:1477896


In [ ]:
df = df.groupby(['preferred_name']).last().reset_index().astype(str)
df.shape

(2474128, 7)

In [ ]:
EFSA = len((df[df.EFSA_PHT != 'None'].EFSA_PHT))
EPPO = len(((df[df.EPPO_CODE != 'None'].EPPO_CODE)))
EPPO_EFSA = len((df[(df.EPPO_CODE != 'None') & (df.EFSA_PHT != 'None')].EFSA_PHT))
NCBI = len((df[(df.EPPO_CODE != 'None') & (df.EFSA_PHT != 'None')].EFSA_PHT))

dbg(EFSA)
dbg(EPPO)
dbg(EPPO_EFSA)

EFSA: 2100
EPPO: 239
EPPO_EFSA: 164


In [ ]:
df.head()

,preferred_name,EPPO_CODE,synonym,ds_language,EFSA_PHT,re,NCBI_TAXID
0,'Abelmoschus esculentus' bunchy top phytoplasma,None,'Abelmoschus esculentus' bunchy top phytoplasma,None,None,None,ncbi:926989
1,'Acacia arabica' phytoplasma,None,'Acacia arabica' phytoplasma,None,None,None,ncbi:2849181
2,'Acacia farnesiana' leaflet phytoplasma,None,'Acacia farnesiana' leaflet phytoplasma,None,None,None,ncbi:2654099
3,'Acacia mangium' little leaf and yellowing phy...,None,'Acacia mangium' little leaf and yellowing phy...,None,None,None,ncbi:2530013
4,'Acacia melanoxylon' phytoplasma,None,'Acacia melanoxylon' phytoplasma,None,None,None,ncbi:1477896


#### Add case information from sibling datasets
Look for elements in list which have case information and add it to lowercase values by replacing them

In [ ]:
def find_indices(list_, element):
    indices = [i for i, x in enumerate(list_) if x == element]
    return indices

In [ ]:
def add_case_info(mix_list):
    low_list = list(map(lambda x: x.lower(),mix_list))
    match_list = [i==j for i, j in zip(mix_list, low_list)]
    for i in tqdm(range(len(mix_list))):
        if match_list[i]:
            low_term = low_list[i]
            candidates = [t for t in mix_list # candidates for substitution are...
                          if (t.lower()==low_term # (1) the same term as the one being substituted (case insensitive)
                              and not match_list[find_indices(mix_list, t)[0]])] # (2) and not lowercase
            #dbg(mix_list[i])
            mix_list[i] = candidates[0] if len(candidates) else mix_list[i]
            #dbg(mix_list[i])
            #print()
    return mix_list

## test
my_list = ['apple', 'Apple', 'banana', 'BANANA', 'Plum', 'plum', 'berries']
add_case_info(my_list)

100%|██████████| 7/7 [00:00<00:00, 27594.11it/s]


['Apple', 'Apple', 'BANANA', 'BANANA', 'Plum', 'Plum', 'berries']

In [ ]:
print(df.shape)
df = df.groupby(['preferred_name']).last().reset_index().astype(str)
print(df.shape)

(2474128, 7)
(2474128, 7)


In [ ]:
df['preferred_name'] = add_case_info(df['preferred_name'])

 94%|█████████▎| 2317271/2474128 [03:06<21:45, 120.11it/s]

In [ ]:
df.to_csv(LR_filepath, index=False)
print(df.shape)

NameError: name 'df' is not defined

### Add RegEx information to EPPO and NCBI lines 

In [ ]:
df = pd.read_csv(LR_filepath).astype(str)

EFSA = len(set(df[df.EFSA_PHT != 'nan'].EFSA_PHT))
EPPO = len((df[df.EPPO_CODE != 'nan'].EPPO_CODE))
EPPO_EFSA = len((df[(df.EPPO_CODE != 'nan') & (df.EFSA_PHT != 'nan')].EFSA_PHT))

dbg(EFSA)
dbg(EPPO)
dbg(EPPO_EFSA)

In [ ]:
## For regex in list of regex:
for pattern, code in tqdm(zip(df['re'], df['EFSA_PHT'])):
    if not isinstance(pattern, str):
        continue
    #try:
    ## For synonym in db:
    for i in range(len(df)):
        ## If there's no regex in line i yet AND there's a match...
        if re.search(str(pattern), str(df['synonym'][i])):   ###df['re'][i] == None and 
            print(df['synonym'][i], pattern)
            df['re'][i] = pattern #...add regex pattern to row in the 're' column 
            df['EFSA_PHT'][i] = code #...add EFSA code to row in the 'EFSA_PHT' column 
            
    #except TypeError:
    #    dbg(type(pattern))

In [ ]:
df.to_csv('re_code_'+LR_filepath, index=False)

# Display data

In [ ]:
df = pd.read_csv('re_code_'+LR_filepath).astype(str)

EFSA = len(set(df[df.EFSA_PHT != 'nan'].EFSA_PHT))
EPPO = len((df[df.EPPO_CODE != 'nan'].EPPO_CODE))
EPPO_EFSA = len((df[(df.EPPO_CODE != 'nan') & (df.EFSA_PHT != 'nan')].EFSA_PHT))

dbg(EFSA)
dbg(EPPO)
dbg(EPPO_EFSA)

NameError: name 'pd' is not defined

In [ ]:
df = df.astype(str)

In [ ]:
df[df.EFSA_PHT != 'nan'].head()

,preferred_name,EPPO_CODE,synonym,ds_language,EFSA_PHT,re,NCBI_TAXID
124,'Borassus aethiopum' palm lethal yellowing phy...,NaN,'Borassus aethiopum' palm lethal yellowing phy...,NaN,PalmLethalYellowingMycoplasm-PHT,palm lethal yellowing phytoplasma,ncbi:1705292
64409,Aleurocanthus camelliae,NaN,camellia spiny whitefly,NaN,Aleurocanthus-PHT,spiny whitefl\w*,ncbi:1000661
64417,Aleurocanthus spiniferus,ALECSN,citrus spiny whitefly,ja,Aleurocanthus-PHT,spiny whitefl\w*,ncbi:593793
65946,Algerian watermelon mosaic virus,NaN,Algerian watermelon mosaic virus,NaN,WatermelonMosaicVirus-PHT,watermelon mosaic virus,ncbi:515575
85805,American plum line pattern virus,APLPV0,American plum line pattern virus,pt,AmericanPlumLinePatternVirus-PHT,plum line pattern virus,ncbi:134632
